In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext line_profiler
import line_profiler

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import HOTS.KmeansCluster as KmeansCluster
from HOTS.Event import Event, SimpleAlphabet
from HOTS.STS import STS
from HOTS.Layer import ClusteringLayer, FilterNHBD
from HOTS.Monitor import DisplayImage, DisplaySurface3D, DisplaySurface2D, DisplayConvergence
from HOTS.Tools import SaveObject, LoadObject
from HOTS.Classifier import Classifier
from HOTS.Network import Network
from HOTS.KmeansCluster import KmeansMaro

In [3]:
tau = 0.02
R = 2
nb_cluster = [4,8,16]
ImageSize = (32,32)

# 1.Profiling the event loading

In [4]:
event_tr, event_te, event_cl, label_tr, label_te = SimpleAlphabet(NbTrainingData=36, NbTestingData=40,
                                                        ClusteringData=[1, 32, 19, 22, 29],
                                                        OutOnePolarity = False, verbose=0,
                                                        Path='../Data/ExtractedStabilized.mat',
                                                        LabelPath='../Data/alphabet_label.pkl')

In [5]:

%lprun -T ProfilingFiles/timing_load_event.txt -f event_te.LoadFromMat -f SimpleAlphabet event_tr, event_te, event_cl, label_tr, label_te = \
                                                        SimpleAlphabet(NbTrainingData=36, NbTestingData=40,\
                                                        ClusteringData=[1, 32, 19, 22, 29],\
                                                        OutOnePolarity = False, verbose=0, \
                                                        Path='../Data/ExtractedStabilized.mat',\
                                                        LabelPath='../Data/alphabet_label.pkl')


*** Profile printout saved to text file 'ProfilingFiles/timing_load_event.txt'. 


In [ ]:
# %load ProfilingFiles/timing_load_event.txt
Timer unit: 1e-06 s

Total time: 0.225514 s
File: /Users/boutin.v/Documents/HOTS/HOTS/Event.py
Function: LoadFromMat at line 34

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    34                                               def LoadFromMat(self,path, image_number, verbose=0):
    35                                                   '''
    36                                                   Load Events from a .mat file. Only the events contained in ListPolarities are kept:
    37                                                   INPUT
    38                                                       + path : a string which is the path of the .mat file (ex : './data_cache/ExtractedStabilized.mat')
    39                                                       + image_number : list with all the numbers of image to load
    40                                                   '''
    41         3       134553  44851.0     59.7          obj = scipy.io.loadmat(path)
    42         3           14      4.7      0.0          ROI = obj['ROI'][0]
    43                                           
    44         3            7      2.3      0.0          if type(image_number) is int:
    45                                                       image_number = [image_number]
    46         3            4      1.3      0.0          elif type(image_number) is not list:
    47                                                       raise TypeError('the type of argument image_number should be int or list')
    48         3            3      1.0      0.0          if verbose>0:
    49                                                       print("chargement des images {0}".format(image_number))
    50         3            2      0.7      0.0          Total_size=0
    51        84           82      1.0      0.0          for idx, each_image  in enumerate(image_number):
    52        81          104      1.3      0.0              image = ROI[each_image][0,0]
    53                                           
    54        81          170      2.1      0.1              Total_size += image[1].shape[1]
    55                                           
    56         3        12339   4113.0      5.5          self.address = np.zeros((Total_size,2)).astype(int)
    57         3          631    210.3      0.3          self.time = np.zeros((Total_size))
    58         3         1484    494.7      0.7          self.polarity = np.zeros((Total_size))
    59         3            4      1.3      0.0          first_idx = 0
    60                                           
    61        84           97      1.2      0.0          for idx, each_image in enumerate(image_number):
    62        81          132      1.6      0.1              image = ROI[each_image][0,0]
    63        81          218      2.7      0.1              last_idx = first_idx + image[0].shape[1]
    64        81         3370     41.6      1.5              self.address[first_idx:last_idx,0] = (image[1] - 1).astype(int)
    65        81         2276     28.1      1.0              self.address[first_idx:last_idx,1] = (image[0] - 1).astype(int)
    66        81         3071     37.9      1.4              self.time[first_idx:last_idx] = (image[3] * 1e-6)
    67        81         2292     28.3      1.0              self.polarity[first_idx:last_idx] = image[2].astype(int)
    68        81           85      1.0      0.0              first_idx = last_idx
    69                                           
    70         3         6672   2224.0      3.0          self.polarity[self.polarity.T==-1] = 0
    71         3         4023   1341.0      1.8          self.polarity = self.polarity.astype(int)
    72                                                   ## Filter only the wanted polarity
    73         3        17362   5787.3      7.7          self.ListPolarities = np.unique(self.polarity)
    74         3         2313    771.0      1.0          filt = np.in1d(self.polarity,np.array(self.ListPolarities))
    75         3        34197  11399.0     15.2          self.filter(filt,mode='itself')
    76         3            9      3.0      0.0          if self.OutOnePolarity == True :
    77                                                       self.polarity = np.ones_like(self.polarity).astype(int)
    78                                                       self.ListPolarities = [1]

Total time: 0.228874 s
File: /Users/boutin.v/Documents/HOTS/HOTS/Event.py
Function: SimpleAlphabet at line 190

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   190                                           def SimpleAlphabet(NbTrainingData,NbTestingData,Path=None,LabelPath=None, ClusteringData=None, OutOnePolarity = False, ListPolarities=None, verbose=0):
   191                                               '''
   192                                               Extract the Data from the SimpleAlphabet DataBase :
   193                                               INPUT :
   194                                                   + NbTrainingData : (int) Number of Training Data
   195                                                   + NbTestingData : (int) Number of Testing Data
   196                                                   + Path : (str) Path of the .mat file. If the path is None, the path is ../database/SimpleAlphabet/ExtractedStabilized.mat
   197                                                   + LabelPath : (str) Path of the .pkl label path. If the path is None, the path is  ../database/SimpleAlphabet/alphabet_label.pkl
   198                                                   + ClusteringData : (list) a list of int indicating the image used to train the cluster. If None, the image used to train the
   199                                                       the cluster are the trainingData
   200                                                   + OutOnePolarity : (bool), transform all polarities into 1 polarity
   201                                                   + ListPolarities : (list), list of the polarity we want to keep
   202                                               OUTPUT :
   203                                                   + event_tr : (<object event>)
   204                                                   + event_te : (<object event>)
   205                                                   + event_cl : (<object event>)
   206                                                   + label_tr :
   207                                                   + label_te :
   208                                               '''
   209         1            4      4.0      0.0      if Path is None :
   210                                                   Path = '../database/SimpleAlphabet/ExtractedStabilized.mat'
   211                                           
   212         1            1      1.0      0.0      if LabelPath is None :
   213                                                   label_list=LoadObject('../database/SimpleAlphabet/alphabet_label.pkl')
   214                                               else :
   215         1          178    178.0      0.1          label_list=LoadObject(LabelPath)
   216                                           
   217         1            2      2.0      0.0      if NbTrainingData+NbTestingData > 76:
   218                                                   raise NameError('Overlaping between TrainingData and Testing Data')
   219         1           19     19.0      0.0      event_tr = Event(ImageSize=(32,32), ListPolarities=ListPolarities, OutOnePolarity=OutOnePolarity)
   220         1            7      7.0      0.0      event_te = Event(ImageSize=(32,32), ListPolarities=ListPolarities, OutOnePolarity=OutOnePolarity)
   221         1            8      8.0      0.0      event_cl = Event(ImageSize=(32,32), ListPolarities=ListPolarities ,OutOnePolarity=OutOnePolarity)
   222         1        82248  82248.0     35.9      event_tr.LoadFromMat(Path,image_number=list(np.arange(0,NbTrainingData)),verbose=verbose)
   223         1        95342  95342.0     41.7      event_te.LoadFromMat(Path,image_number=list(np.arange(NbTrainingData, NbTrainingData+NbTestingData)),verbose=verbose)
   224                                           
   225         1            2      2.0      0.0      if ClusteringData is None :
   226                                                   event_cl = event_tr
   227                                               else :
   228         1        49969  49969.0     21.8          event_cl.LoadFromMat(Path,image_number=ClusteringData,verbose=verbose)
   229                                           
   230                                               ## Generate Groud Truth Label
   231        37           46      1.2      0.0      for idx,img in enumerate(np.arange(0,NbTrainingData)):
   232        36           30      0.8      0.0          if idx!= 0:
   233        35          468     13.4      0.2              label_tr = np.vstack((label_tr,label_list[img][0]))
   234                                                   else:
   235         1            2      2.0      0.0              label_tr = label_list[img][0]
   236                                           
   237        41           40      1.0      0.0      for idx,img in enumerate(np.arange(NbTrainingData,NbTrainingData+NbTestingData)):
   238        40           29      0.7      0.0          if idx!= 0:
   239        39          477     12.2      0.2              label_te = np.vstack((label_te,label_list[img][0]))
   240                                                   else:
   241         1            1      1.0      0.0              label_te = label_list[img][0]
   242                                           
   243         1            1      1.0      0.0      return event_tr, event_te, event_cl, label_tr, label_te

# 2.Profiling the Network Training (with KmeansLagorce Algo)

In [7]:
InputFilter = FilterNHBD(threshold=15,neighbourhood=2)
L1 = ClusteringLayer(tau=tau, R=R, verbose=1, ThrFilter=0, LearningAlgo='lagorce',
                         kernel='exponential',eta=None, eta_homeo=None, sigma=None)
L2 = ClusteringLayer(tau=10*tau, R=2*R, verbose=1, ThrFilter=0, LearningAlgo='lagorce'
                         ,kernel='exponential',eta=None, eta_homeo=None, sigma=None)
L3 = ClusteringLayer(tau=10*10*tau, R=2*2*R, verbose=1, ThrFilter=0, LearningAlgo='lagorce'
                         ,kernel='exponential',eta=None, eta_homeo=None, sigma=None)
Net = Network([InputFilter,L1,L2,L3])

In [8]:
'''
%lprun -T ProfilingFiles/timing_NetworkTraining_v0.txt -f Net.TrainCluster -f L1.TrainLayer -f L1.ClusterLayer.fit\
                                -f L1.ClusterLayer.predict \
                                ClusterLayer,event_output = \
                                Net.TrainCluster(event = event_cl, NbClusterList = nb_cluster,\
                                record_each=100, NbCycle=1)
'''

'\n%lprun -T ProfilingFiles/timing_NetworkTraining_v0.txt -f Net.TrainCluster -f L1.TrainLayer -f L1.ClusterLayer.fit                                -f L1.ClusterLayer.predict                                 ClusterLayer,event_output =                                 Net.TrainCluster(event = event_cl, NbClusterList = nb_cluster,                                record_each=100, NbCycle=1)\n'

In [10]:
%lprun -T ProfilingFiles/timing_NetworkTraining_v1.txt -f Net.TrainCluster -f L1.TrainLayer \
                                -f L1.ClusterLayer.fit -f KmeansCluster.predict_jit \
                                -f KmeansCluster.jitted_prediction\
                                ClusterLayer,event_output = \
                                Net.TrainCluster(event = event_cl, NbClusterList = nb_cluster,\
                                to_record=True, NbCycle=1)

Generation of SpatioTemporal Surface in ------ 1.58 s
Clustering SpatioTemporal Surface in ------ 7.50 s
Generation of SpatioTemporal Surface in ------ 1.91 s
Clustering SpatioTemporal Surface in ------ 6.71 s
Generation of SpatioTemporal Surface in ------ 3.63 s
Clustering SpatioTemporal Surface in ------ 21.15 s

*** Profile printout saved to text file 'ProfilingFiles/timing_NetworkTraining_v1.txt'. 


In [ ]:
# %load ProfilingFiles/timing_NetworkTraining_v1.txt
Timer unit: 1e-06 s

Total time: 32.3982 s
File: /Users/boutin.v/Documents/HOTS/HOTS/KmeansCluster.py
Function: fit at line 85

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    85                                               def fit (self,STS, init=None, NbCycle=1):
    86                                                   '''
    87                                                   Methods to learn prototypes fitting data
    88                                                   INPUT :
    89                                                       + STS : (<STS object>) Stream of SpatioTemporal Surface to fit
    90                                                       + init : (<string>) Method to initialize the prototype ('rdn' or None)
    91                                                       + NbCycle : (<int>) Number of time the stream is going to be browse.
    92                                                   OUTPUT :
    93                                                       + prototype : (<np.array>) matrix of size (nb_cluster,nb_polarity*((2*R+1)*(2*R+1)))
    94                                                           representing the centers of clusters
    95                                                   '''
    96         3           12      4.0      0.0          tic = time.time()
    97         3       634338 211446.0      2.0          surface = STS.Surface.copy()
    98         3           15      5.0      0.0          if self.to_record == True :
    99         3           18      6.0      0.0              self.record_each = surface.shape[0]//100
   100         3            3      1.0      0.0          if init is None :
   101         3           34     11.3      0.0              self.prototype=surface[:self.nb_cluster,:]
   102                                                   elif init == 'rdn' :
   103                                                       idx = np.random.permutation(np.arange(surface.shape[0]))[:self.nb_cluster]
   104                                                       self.prototype = surface[idx, :]
   105                                                   else :
   106                                                       raise NameError('argument '+str(init)+' is not valid. Only None or rdn are valid')
   107         3            3      1.0      0.0          idx_global=0
   108         3           47     15.7      0.0          nb_proto = np.zeros((self.nb_cluster)).astype(int)
   109         6           18      3.0      0.0          for each_cycle in range(NbCycle):
   110         3           11      3.7      0.0              nb_proto = np.zeros((self.nb_cluster)).astype(int)
   111    176982       261219      1.5      0.8              for idx, Si in enumerate(surface):
   112    176979      9319500     52.7     28.8                  Distance_to_proto = np.linalg.norm(Si - self.prototype,ord=2,axis=1)
   113    176979       900310      5.1      2.8                  closest_proto_idx = np.argmin(Distance_to_proto)
   114    176979       264249      1.5      0.8                  pk = nb_proto[closest_proto_idx]
   115    176979       328179      1.9      1.0                  Ck = self.prototype[closest_proto_idx,:]
   116    176979       370571      2.1      1.1                  alpha = 0.01/(1+pk/20000)
   117    176979      1640830      9.3      5.1                  beta = np.dot(Ck, Si)/(np.sqrt(np.dot(Si, Si))*np.sqrt(np.dot(Ck, Ck)))
   118    176979      1577769      8.9      4.9                  Ck_t = Ck + alpha*(Si - beta*Ck)
   119                                                           #Ck_t = Ck + alpha*beta*(Si - Ck)
   120    176979       359700      2.0      1.1                  nb_proto[closest_proto_idx] += 1
   121    176979       464389      2.6      1.4                  self.prototype[closest_proto_idx, :] = Ck_t
   122                                           
   123    176979       224709      1.3      0.7                  if self.to_record == True :
   124    176979       316215      1.8      1.0                      if idx_global % int(self.record_each) == 0 :
   125                                                                   #self.monitor(STS,SurfaceFilter=1000)
   126       303     14693807  48494.4     45.4                          pol, output_distance, = self.predict(STS,SurfaceFilter=1000)
   127       303        20115     66.4      0.1                          error = np.mean(output_distance)
   128       303        81691    269.6      0.3                          active_probe = np.histogram(pol,bins=np.arange(self.nb_cluster+1))[0]
   129       303          811      2.7      0.0                          record_one = pd.DataFrame([{'error':error,
   130       303          468      1.5      0.0                                                      'histo':active_probe}
   131                                                                                               ],
   132       303       333155   1099.5      1.0                                              index=[idx_global])
   133       303       397476   1311.8      1.2                          self.record = pd.concat([self.record, record_one])
   134                                           
   135    176979       208138      1.2      0.6                  idx_global += 1
   136         3            8      2.7      0.0              tac = time.time()
   137         3            5      1.7      0.0          self.nb_proto = nb_proto
   138         3            3      1.0      0.0          if self.verbose > 0:
   139         3          389    129.7      0.0              print('Clustering SpatioTemporal Surface in ------ {0:.2f} s'.format(tac-tic))
   140                                           
   141         3            4      1.3      0.0          return self.prototype

Total time: 0 s
File: /Users/boutin.v/Documents/HOTS/HOTS/KmeansCluster.py
Function: predict_jit at line 297

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   297                                           def predict_jit(Surface, prototype, nb_cluster, event=None, SurfaceFilter=None):
   298                                               '''
   299                                               Methods to predict the closest prototype from a stream a STS
   300                                               INPUT :
   301                                                   + STS : (<STS object>) Stream of SpatioTemporal Surface to fit
   302                                                   + event : (<event object>) event associated to the STS. return another event stream with new polarity
   303                                                   + SurfaceFilter : (<int>) To predict only on a small subsample of Surface of size (SurfaceFilter)
   304                                               OUTPUT :
   305                                                   + output_distance : (<np.array>)
   306                                                   + event_output : (<event.object>)
   307                                                   + polarity : (<np.array>)
   308                                               '''
   309                                               if SurfaceFilter == None:
   310                                                   to_predict = Surface
   311                                               else :
   312                                                   random_selection = np.random.permutation(np.arange(Surface.shape[0]))[:SurfaceFilter]
   313                                                   to_predict = Surface[random_selection]
   314                                           
   315                                               #if self.prototype is None :
   316                                               #    raise ValueError('Train the Cluster before doing prediction')
   317                                               #polarity,output_distance = np.zeros(to_predict.shape[0]).astype(int),np.zeros(to_predict.shape[0])
   318                                           
   319                                               polarity, output_distance = jitted_prediction(to_predict,prototype)
   320                                               #for idx,surface in enumerate(to_predict):
   321                                               #    Euclidian_distance = np.linalg.norm(surface-prototype,ord=2,axis=1)
   322                                               #    polarity[idx] = np.argmin(Euclidian_distance)
   323                                               #    output_distance[idx] = np.amin(Euclidian_distance)
   324                                               if event is not None :
   325                                                   event_output = event.copy()
   326                                                   event_output.polarity = polarity
   327                                                   event_output.ListPolarities= list(np.arange(nb_cluster))
   328                                                   return event_output, output_distance
   329                                           
   330                                               return output_distance,polarity

Total time: 0 s
File: /Users/boutin.v/Documents/HOTS/HOTS/KmeansCluster.py
Function: jitted_prediction at line 332

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   332                                           @jit(nopython=True)
   333                                           def jitted_prediction(to_predict,prototype):
   334                                               polarity,output_distance = np.zeros(to_predict.shape[0]),np.zeros(to_predict.shape[0])
   335                                               for idx in range(to_predict.shape[0]):
   336                                                   Euclidian_distance = np.sqrt(np.sum((to_predict[idx] - prototype)**2,axis=1))
   337                                                   polarity[idx] = np.argmin(Euclidian_distance)
   338                                                   output_distance[idx] = np.amin(Euclidian_distance)
   339                                               return output_distance,polarity

Total time: 50.0564 s
File: /Users/boutin.v/Documents/HOTS/HOTS/Layer.py
Function: TrainLayer at line 116

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   116                                               def TrainLayer(self, event, nb_cluster, to_record=False, NbCycle=1):
   117                                                   '''
   118                                                   Learn the Cluster
   119                                                   INPUT :
   120                                                       + event (<object event>) : input event
   121                                                       + nb_cluster(<int>) : nb of centers
   122                                                       + record_each (<int>) : record the convergence error each reach_each
   123                                                       + NbCycle (<int>) : number of time we repeat the learning process. Need to be used when not enought training data to reach the convergence
   124                                           
   125                                                   OUTPUT :
   126                                                       + output (<object event>) : Event with new polarities corresponding to the closest cluster center
   127                                                       + ClusterLayer (<object Cluster) : Learnt cluster
   128                                                   '''
   129         3            6      2.0      0.0          self.input=event
   130         3          630    210.0      0.0          self.SpTe_Layer = STS(tau=self.tau, R=self.R, verbose=self.verbose, sigma=self.sigma)
   131         3      7122620 2374206.7     14.2          Surface_Layer = self.SpTe_Layer.create(event = self.input, kernel=self.kernel)
   132         3       719359 239786.3      1.4          event_filtered, filt = self.SpTe_Layer.FilterRecent(event = self.input, threshold=self.ThrFilter)
   133         3           14      4.7      0.0          self.ClusterLayer.nb_cluster, self.ClusterLayer.to_record = nb_cluster, to_record
   134         3     35351943 11783981.0     70.6          Prototype = self.ClusterLayer.fit(self.SpTe_Layer, NbCycle=NbCycle)
   135         3      6861839 2287279.7     13.7          self.output,_ = self.ClusterLayer.predict(STS=self.SpTe_Layer,event = event_filtered)
   136                                           
   137         3            7      2.3      0.0          return self.output, self.ClusterLayer

Total time: 51.61 s
File: /Users/boutin.v/Documents/HOTS/HOTS/Network.py
Function: TrainCluster at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     8                                               def TrainCluster(self, event, NbClusterList, to_record=False, NbCycle=1, KN=None, nb_cluster=None ) :
     9         1            2      2.0      0.0          event_i = event
    10         1            1      1.0      0.0          idx_Layer = 0
    11         1            1      1.0      0.0          ClusterList = list()
    12         5           10      2.0      0.0          for idx, each_Layer in enumerate(self.Layers):
    13         4            8      2.0      0.0              if each_Layer.type == 'void':
    14                                                           print('problem !!' )
    15         4            3      0.8      0.0              elif each_Layer.type == 'Filter':
    16         1      1505929 1505929.0      2.9                  event_o = each_Layer.RunLayer(event_i)
    17         3            2      0.7      0.0              elif each_Layer.type == 'Layer':
    18                                                           #event_o, Cluster = each_Layer.TrainLayer(event_i, nb_cluster*(KN)**(idx_Layer), record_each=record_each, NbCycle=NbCycle)
    19                                                           #print('nb_cluster at layer {0} :  {1}'.format(idx_Layer+1,NbClusterList[idx_Layer]))
    20         3     50104031 16701343.7     97.1                  event_o, Cluster = each_Layer.TrainLayer(event_i, NbClusterList[idx_Layer], to_record=to_record, NbCycle=NbCycle)
    21         3           10      3.3      0.0                  ClusterList.append(Cluster)
    22         3            3      1.0      0.0                  idx_Layer = idx_Layer + 1
    23                                                           #if self.verbose>0 :
    24                                                           #    print('Training of Layer {0}'.format(idx_Layer))
    25                                                       else :
    26                                                           print(type(each_Layer))
    27         4            5      1.2      0.0              event_i = event_o
    28                                                       #print(event_i.address.shape[0])
    29         1            1      1.0      0.0          return ClusterList, event_o

In [ ]:
# %load ProfilingFiles/timing_NetworkTraining_v0.txt
Timer unit: 1e-06 s

Total time: 155.346 s
File: /Users/boutin.v/Documents/HOTS/HOTS/KmeansCluster.py
Function: fit at line 39

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    39                                               def fit (self,STS, init=None, NbCycle=1):
    40                                                   '''
    41                                                   Methods to learn prototypes fitting data
    42                                                   INPUT :
    43                                                       + STS : (<STS object>) Stream of SpatioTemporal Surface to fit
    44                                                       + init : (<string>) Method to initialize the prototype ('rdn' or None)
    45                                                       + NbCycle : (<int>) Number of time the stream is going to be browse.
    46                                                   OUTPUT :
    47                                                       + prototype : (<np.array>) matrix of size (nb_cluster,nb_polarity*((2*R+1)*(2*R+1)))
    48                                                           representing the centers of clusters
    49                                                   '''
    50         3           12      4.0      0.0          tic = time.time()
    51                                           
    52         3       736927 245642.3      0.5          surface = STS.Surface.copy()
    53                                           
    54         3           17      5.7      0.0          if init is None :
    55         3           42     14.0      0.0              self.prototype=surface[:self.nb_cluster,:]
    56                                                   elif init == 'rdn' :
    57                                                       idx = np.random.permutation(np.arange(surface.shape[0]))[:self.nb_cluster]
    58                                                       self.prototype = surface[idx, :]
    59                                                   else :
    60                                                       raise NameError('argument '+str(init)+' is not valid. Only None or rdn are valid')
    61         3            4      1.3      0.0          idx_global=0
    62         3           54     18.0      0.0          nb_proto = np.zeros((self.nb_cluster)).astype(int)
    63         6           25      4.2      0.0          for each_cycle in range(NbCycle):
    64         3           29      9.7      0.0              nb_proto = np.zeros((self.nb_cluster)).astype(int)
    65    176982       268926      1.5      0.2              for idx, Si in enumerate(surface):
    66                                                           #Distance_to_proto = EuclidianNorm(Si, self.prototype)
    67    176979      9890860     55.9      6.4                  Distance_to_proto = np.linalg.norm(Si - self.prototype,ord=2,axis=1)
    68    176979       929999      5.3      0.6                  closest_proto_idx = np.argmin(Distance_to_proto)
    69    176979       264382      1.5      0.2                  pk = nb_proto[closest_proto_idx]
    70    176979       339880      1.9      0.2                  Ck = self.prototype[closest_proto_idx,:]
    71    176979       381220      2.2      0.2                  alpha = 0.01/(1+pk/20000)
    72    176979      1687479      9.5      1.1                  beta = np.dot(Ck, Si)/(np.sqrt(np.dot(Si, Si))*np.sqrt(np.dot(Ck, Ck)))
    73    176979      1658392      9.4      1.1                  Ck_t = Ck + alpha*(Si - beta*Ck)
    74                                                           #Ck_t = (1 - alpha*beta) * Ck + alpha*beta*Si
    75    176979       362253      2.0      0.2                  nb_proto[closest_proto_idx] += 1
    76                                                           #Ck_t /= np.amax(Ck_t)
    77    176979       463660      2.6      0.3                  self.prototype[closest_proto_idx, :] = Ck_t
    78                                           
    79    176979       238358      1.3      0.2                  if self.record_each != 0 :
    80    176979       311850      1.8      0.2                      if idx_global % int(self.record_each) == 0 :
    81      1770    132669504  74954.5     85.4                          output_distance,pol = self.predict(STS,SurfaceFilter=1000)
    82                                                                   #output_distance,pol = predict_jit(Surface = STS.Surface, prototype=self.prototype,
    83                                                                   #                        nb_cluster=self.nb_cluster,SurfaceFilter=1000)
    84      1770        88222     49.8      0.1                          error = np.mean(output_distance)
    85      1770       470504    265.8      0.3                          active_probe = np.histogram(pol,bins=np.arange(self.nb_cluster+1))[0]
    86      1770         4719      2.7      0.0                          record_one = pd.DataFrame([{'error':error,
    87      1770         2569      1.5      0.0                                                      'histo':active_probe}
    88                                                                                               ],
    89      1770      1967091   1111.4      1.3                                              index=[idx_global])
    90      1770      2395465   1353.4      1.5                          self.record = pd.concat([self.record, record_one])
    91                                           
    92    176979       213021      1.2      0.1                  idx_global += 1
    93         3           11      3.7      0.0              tac = time.time()
    94                                                   #self.prototype = prototype
    95         3            8      2.7      0.0          self.nb_proto = nb_proto
    96         3            5      1.7      0.0          if self.verbose > 0:
    97         3          386    128.7      0.0              print('Clustering SpatioTemporal Surface in ------ {0:.2f} s'.format(tac-tic))
    98                                           
    99         3            6      2.0      0.0          return self.prototype

Total time: 136.833 s
File: /Users/boutin.v/Documents/HOTS/HOTS/KmeansCluster.py
Function: predict at line 101

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   101                                               def predict(self, STS, event=None, SurfaceFilter=None):
   102                                                   '''
   103                                                   Methods to predict the closest prototype from a stream a STS
   104                                                   INPUT :
   105                                                       + STS : (<STS object>) Stream of SpatioTemporal Surface to fit
   106                                                       + event : (<event object>) event associated to the STS. return another event stream with new polarity
   107                                                       + SurfaceFilter : (<int>) To predict only on a small subsample of Surface of size (SurfaceFilter)
   108                                                   OUTPUT :
   109                                                       + output_distance : (<np.array>)
   110                                                       + event_output : (<event.object>)
   111                                                       + polarity : (<np.array>)
   112                                                   '''
   113      1773         1859      1.0      0.0          if SurfaceFilter == None:
   114         3            6      2.0      0.0              to_predict = STS.Surface
   115                                                   else :
   116      1770      2049950   1158.2      1.5              random_selection = np.random.permutation(np.arange(STS.Surface.shape[0]))[:SurfaceFilter]
   117      1770      4255486   2404.2      3.1              to_predict = STS.Surface[random_selection]
   118                                           
   119      1773         6290      3.5      0.0          if self.prototype is None :
   120                                                       raise ValueError('Train the Cluster before doing prediction')
   121      1773        41762     23.6      0.0          polarity,output_distance = np.zeros(to_predict.shape[0]).astype(int),np.zeros(to_predict.shape[0])
   122                                           
   123   1948752      2345789      1.2      1.7          for idx,surface in enumerate(to_predict):
   124   1946979    103452160     53.1     75.6              Euclidian_distance = np.linalg.norm(surface-self.prototype,ord=2,axis=1)
   125   1946979      9355867      4.8      6.8              polarity[idx] = np.argmin(Euclidian_distance)
   126   1946979     15318331      7.9     11.2              output_distance[idx] = np.amin(Euclidian_distance)
   127      1773         1653      0.9      0.0          if event is not None :
   128         3         2295    765.0      0.0              event_output = event.copy()
   129         3           13      4.3      0.0              event_output.polarity = polarity
   130         3           39     13.0      0.0              event_output.ListPolarities= list(np.arange(self.nb_cluster))
   131         3            3      1.0      0.0              return event_output, output_distance
   132                                                   else :
   133      1770         1285      0.7      0.0              return output_distance,polarity

Total time: 178.483 s
File: /Users/boutin.v/Documents/HOTS/HOTS/Layer.py
Function: TrainLayer at line 116

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   116                                               def TrainLayer(self, event, nb_cluster, record_each=0, NbCycle=1):
   117                                                   '''
   118                                                   Learn the Cluster
   119                                                   INPUT :
   120                                                       + event (<object event>) : input event
   121                                                       + nb_cluster(<int>) : nb of centers
   122                                                       + record_each (<int>) : record the convergence error each reach_each
   123                                                       + NbCycle (<int>) : number of time we repeat the learning process. Need to be used when not enought training data to reach the convergence
   124                                           
   125                                                   OUTPUT :
   126                                                       + output (<object event>) : Event with new polarities corresponding to the closest cluster center
   127                                                       + ClusterLayer (<object Cluster) : Learnt cluster
   128                                                   '''
   129         3            3      1.0      0.0          self.input=event
   130         3          530    176.7      0.0          self.SpTe_Layer = STS(tau=self.tau, R=self.R, verbose=self.verbose, sigma=self.sigma)
   131         3      7147126 2382375.3      4.0          Surface_Layer = self.SpTe_Layer.create(event = self.input, kernel=self.kernel)
   132         3       834896 278298.7      0.5          event_filtered, filt = self.SpTe_Layer.FilterRecent(event = self.input, threshold=self.ThrFilter)
   133         3           11      3.7      0.0          self.ClusterLayer.nb_cluster, self.ClusterLayer.record_each = nb_cluster, record_each
   134                                                   #if self.LearningAlgo == 'lagorce' :
   135                                                   #    self.ClusterLayer = KmeansLagorce(nb_cluster = nb_cluster,verbose=self.verbose, record_each=record_each)
   136                                                   #elif self.LearningAlgo == 'maro' :
   137                                                   #    self.ClusterLayer = KmeansMaro(nb_cluster = nb_cluster,verbose=self.verbose, record_each=record_each,
   138                                                   #                                eta=self.eta)
   139                                                   #elif self.LearningAlgo == 'homeo' :
   140                                                   #    self.ClusterLayer = KmeansHomeo(nb_cluster = nb_cluster,verbose=self.verbose, record_each=record_each,
   141                                                   #                                eta=self.eta, eta_homeo=self.eta_homeo)
   142         3    158318914 52772971.3     88.7          Prototype = self.ClusterLayer.fit(self.SpTe_Layer, NbCycle=NbCycle)
   143         3     12181575 4060525.0      6.8          self.output,_ = self.ClusterLayer.predict(STS=self.SpTe_Layer,event = event_filtered)
   144                                           
   145         3            3      1.0      0.0          return self.output, self.ClusterLayer

Total time: 180.092 s
File: /Users/boutin.v/Documents/HOTS/HOTS/Network.py
Function: TrainCluster at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     8                                               def TrainCluster(self, event, NbClusterList, record_each=0, NbCycle=1, KN=None, nb_cluster=None ) :
     9         1            2      2.0      0.0          event_i = event
    10         1            1      1.0      0.0          idx_Layer = 0
    11         1            2      2.0      0.0          ClusterList = list()
    12         5            9      1.8      0.0          for idx, each_Layer in enumerate(self.Layers):
    13         4            7      1.8      0.0              if each_Layer.type == 'void':
    14                                                           print('problem !!' )
    15         4            3      0.8      0.0              elif each_Layer.type == 'Filter':
    16         1      1490550 1490550.0      0.8                  event_o = each_Layer.RunLayer(event_i)
    17         3            1      0.3      0.0              elif each_Layer.type == 'Layer':
    18                                                           #event_o, Cluster = each_Layer.TrainLayer(event_i, nb_cluster*(KN)**(idx_Layer), record_each=record_each, NbCycle=NbCycle)
    19                                                           #print('nb_cluster at layer {0} :  {1}'.format(idx_Layer+1,NbClusterList[idx_Layer]))
    20         3    178601413 59533804.3     99.2                  event_o, Cluster = each_Layer.TrainLayer(event_i, NbClusterList[idx_Layer], record_each=record_each, NbCycle=NbCycle)
    21         3           10      3.3      0.0                  ClusterList.append(Cluster)
    22         3            3      1.0      0.0                  idx_Layer = idx_Layer + 1
    23                                                           #if self.verbose>0 :
    24                                                           #    print('Training of Layer {0}'.format(idx_Layer))
    25                                                       else :
    26                                                           print(type(each_Layer))
    27         4            3      0.8      0.0              event_i = event_o
    28                                                       #print(event_i.address.shape[0])
    29         1            1      1.0      0.0          return ClusterList, event_o

# 3.Profiling the Network Running

In [12]:
#%lprun -T ProfilingFiles/timing_NetworkRunning.txt -f Net.RunNetwork -f L1.RunLayer\
#                event_o_tr = Net.RunNetwork(event_tr, ClusterList=ClusterLayer)

In [13]:
%lprun -T ProfilingFiles/timing_NetworkRunning_v1.txt -f Net.RunNetwork -f L1.RunLayer\
                event_o_tr = Net.RunNetwork(event_tr, ClusterList=ClusterLayer)

Generation of SpatioTemporal Surface in ------ 9.87 s
Generation of SpatioTemporal Surface in ------ 12.09 s
Generation of SpatioTemporal Surface in ------ 26.04 s

*** Profile printout saved to text file 'ProfilingFiles/timing_NetworkRunning_v1.txt'. 


In [ ]:
# %load ProfilingFiles/timing_NetworkRunning_v1.txt
Timer unit: 1e-06 s

Total time: 122.379 s
File: /Users/boutin.v/Documents/HOTS/HOTS/Layer.py
Function: RunLayer at line 100

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   100                                               def RunLayer(self, event, Cluster):
   101                                                   '''
   102                                                   Associate each polarity of the event input to the prototype of the cluster
   103                                                   INPUT :
   104                                                       + event (<object Event>) : input event
   105                                                       + Cluster (<object Cluster>) : Cluster, previously trained,
   106                                                   OUTPUT :
   107                                                       + self.output : (<object Event>) : Event with new polarities corresponding to the closest cluster center
   108                                                   '''
   109         3          793    264.3      0.0          self.input = event
   110         3        52550  17516.7      0.0          self.SpTe_Layer = STS(tau=self.tau, R=self.R, verbose=self.verbose,sigma=self.sigma)
   111         3     48010008 16003336.0     39.2          Surface_Layer = self.SpTe_Layer.create(event=self.input, kernel=self.kernel)
   112         3     17168304 5722768.0     14.0          event_filtered, filt = self.SpTe_Layer.FilterRecent(event = self.input, threshold=self.ThrFilter) ## Check that THRFilter=0 is equivalent to no Filter
   113         3     57147523 19049174.3     46.7          self.output,_ = Cluster.predict(STS=self.SpTe_Layer,event = event_filtered)
   114         3           17      5.7      0.0          return self.output

Total time: 133.14 s
File: /Users/boutin.v/Documents/HOTS/HOTS/Network.py
Function: RunNetwork at line 31

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    31                                               def RunNetwork(self,event, ClusterList):
    32         1            1      1.0      0.0          event_i = event
    33         1            0      0.0      0.0          idx_Layer = 0
    34         5         1254    250.8      0.0          for idx, each_Layer in enumerate(self.Layers):
    35         4            8      2.0      0.0              if each_Layer.type == 'void':
    36                                                           print('problem !!' )
    37         4            2      0.5      0.0              elif each_Layer.type == 'Filter':
    38         1     10237561 10237561.0      7.7                  event_o = each_Layer.RunLayer(event_i)
    39         3            2      0.7      0.0              elif each_Layer.type == 'Layer':
    40         3    122900891 40966963.7     92.3                  event_o = each_Layer.RunLayer(event_i, Cluster = ClusterList[idx_Layer] )
    41         3           20      6.7      0.0                  idx_Layer = idx_Layer + 1
    42         3           28      9.3      0.0                  if self.verbose>0:
    43                                                               print('Layer')
    44                                                       else :
    45                                                           print(type(each_Layer))
    46         4            5      1.2      0.0              event_i = event_o
    47         1            4      4.0      0.0          return event_o

In [ ]:
# %load ProfilingFiles/timing_NetworkRunning.txt
Timer unit: 1e-06 s

Total time: 155.641 s
File: /Users/boutin.v/Documents/HOTS/HOTS/Layer.py
Function: RunLayer at line 100

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   100                                               def RunLayer(self, event, Cluster):
   101                                                   '''
   102                                                   Associate each polarity of the event input to the prototype of the cluster
   103                                                   INPUT :
   104                                                       + event (<object Event>) : input event
   105                                                       + Cluster (<object Cluster>) : Cluster, previously trained,
   106                                                   OUTPUT :
   107                                                       + self.output : (<object Event>) : Event with new polarities corresponding to the closest cluster center
   108                                                   '''
   109         3          740    246.7      0.0          self.input = event
   110         3       118292  39430.7      0.1          self.SpTe_Layer = STS(tau=self.tau, R=self.R, verbose=self.verbose,sigma=self.sigma)
   111         3     50567125 16855708.3     32.5          Surface_Layer = self.SpTe_Layer.create(event=self.input, kernel=self.kernel)
   112         3     15658752 5219584.0     10.1          event_filtered, filt = self.SpTe_Layer.FilterRecent(event = self.input, threshold=self.ThrFilter) ## Check that THRFilter=0 is equivalent to no Filter
   113         3     89296003 29765334.3     57.4          self.output,_ = Cluster.predict(STS=self.SpTe_Layer,event = event_filtered)
   114         3            7      2.3      0.0          return self.output

Total time: 166.859 s
File: /Users/boutin.v/Documents/HOTS/HOTS/Network.py
Function: RunNetwork at line 31

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    31                                               def RunNetwork(self,event, ClusterList):
    32         1            3      3.0      0.0          event_i = event
    33         1            1      1.0      0.0          idx_Layer = 0
    34         5         2078    415.6      0.0          for idx, each_Layer in enumerate(self.Layers):
    35         4            7      1.8      0.0              if each_Layer.type == 'void':
    36                                                           print('problem !!' )
    37         4            3      0.8      0.0              elif each_Layer.type == 'Filter':
    38         1     10496166 10496166.0      6.3                  event_o = each_Layer.RunLayer(event_i)
    39         3            2      0.7      0.0              elif each_Layer.type == 'Layer':
    40         3    156358989 52119663.0     93.7                  event_o = each_Layer.RunLayer(event_i, Cluster = ClusterList[idx_Layer] )
    41         3           20      6.7      0.0                  idx_Layer = idx_Layer + 1
    42         3         1288    429.3      0.0                  if self.verbose>0:
    43                                                               print('Layer')
    44                                                       else :
    45                                                           print(type(each_Layer))
    46         4            9      2.2      0.0              event_i = event_o
    47         1            5      5.0      0.0          return event_o

# 4.Profiling the Classification

In [14]:
event_o_te = Net.RunNetwork(event_te, ClusterList=ClusterLayer)
Classif0 = Classifier(event_o_tr,event_o_te,TrainingLabel=label_tr, GroundTruth=label_te)


Generation of SpatioTemporal Surface in ------ 8.19 s
Generation of SpatioTemporal Surface in ------ 10.62 s
Generation of SpatioTemporal Surface in ------ 25.68 s


In [23]:

%lprun -T ProfilingFiles/timing_NetworkClassifier.txt -f Classif0.HistogramDistance \
                prediction, accuracy, method = Classif0.HistogramDistance(to_print=True)

battacha:0.85% ### normalized:0.825% ### euclidian:0.85% ### 

*** Profile printout saved to text file 'ProfilingFiles/timing_NetworkClassifier.txt'. 


In [ ]:
# %load ProfilingFiles/timing_NetworkClassifier.txt
Timer unit: 1e-06 s

Total time: 0.051973 s
File: /Users/boutin.v/Documents/HOTS/HOTS/Classifier.py
Function: HistogramDistance at line 41

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    41                                               def HistogramDistance(self,methods=['euclidian','normalized','battacha'], to_print=False):#(histo_to_classify,proto_histo,proto_label_list=None):
    42                                                   '''
    43                                                   method to classify using histogram distance between prototypes and :
    44                                                   INPUT :
    45                                                       + methods : (<list>) of (<string>) to inform which norm to use to calculate histogram distances
    46                                                           should be euclidian, normalized or battacha
    47                                                       + to_print : (<boolean>) to indicate if we want to print to resulting accuracy
    48                                                   OUTPUT :
    49                                                       + prediction : (<list>) of (<np.array>) of size (nb_of_testing_sample,1) representing the predicted output
    50                                                           for each method
    51                                                       + accu : (<list>) of (<float>) of size (nb_of_methods) representing the accuracy on the testing_sample for each method
    52                                                       + methods : (<list>) of (<string>) of size (nb_of_methods) representing the name of the method used to calculate distance
    53                                                   '''
    54         1        26339  26339.0     50.7          histo_train, pola_train = GenerateHistogram(self.event_train)
    55         1        17151  17151.0     33.0          histo_test, pola_test = GenerateHistogram(self.event_test)
    56         1            3      3.0      0.0          prediction = list()
    57         1            0      0.0      0.0          allmethod = list()
    58         4            5      1.2      0.0          for each_method in methods:
    59         3           58     19.3      0.1              output = np.zeros((histo_test.shape[0],1)).astype(np.str_)
    60       123          183      1.5      0.4              for idx,each_histo in enumerate(histo_test):
    61                                           
    62       120         6179     51.5     11.9                  distance = Norm(each_histo,histo_train,each_method)
    63       120          797      6.6      1.5                  min_dist = np.argmin(distance)
    64       120          444      3.7      0.9                  output[idx,0] = self.TrainingLabel[min_dist][0]
    65                                           
    66         3            5      1.7      0.0              prediction.append(output)
    67         3            3      1.0      0.0              allmethod.append(each_method)
    68         1          661    661.0      1.3          accu = self.Accuracy(prediction)
    69         1            2      2.0      0.0          if to_print==True:
    70         1            1      1.0      0.0              to_write=''
    71         4            6      1.5      0.0              for each_accu, each_method in zip(accu,allmethod):
    72                                                           #print(each_accu))
    73                                                           #print(type(each_method))
    74         3           20      6.7      0.0                  to_write = str(each_method) + ':' + str(each_accu%100) + '% ### '+ to_write
    75         1          115    115.0      0.2              print(to_write)
    76                                           
    77         1            1      1.0      0.0          return prediction, accu, allmethod